In [1]:
import random
from data import ImageDetectionsField, TextField, RawField
from data import COCO, DataLoader
from data.dataset import AP_Dataset, APeval_Dataset, SA_Dataset, SAeval_Dataset
import evaluation
from evaluation import PTBTokenizer, Cider
# from models.transformer import Transformer, MemoryAugmentedEncoder, MeshedDecoder, ScaledDotProductAttentionMemory
from models.grid_m2 import Transformer, TransformerEncoder, MeshedDecoder, ScaledDotProductAttentionMemory, ScaledDotProductAttention
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.nn import NLLLoss
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader as TorchDataLoader
from torch.utils.data import Dataset as TorchDataset
import argparse, os, pickle
from tqdm import tqdm
import numpy as np
import itertools
import multiprocessing
from shutil import copyfile
import h5py
from utils import text_progress2, text_progress
import pandas as pd
import json

random.seed(1234)
torch.manual_seed(1234)
np.random.seed(1234)

/usr/local/miniconda3/envs/m2release/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/miniconda3/envs/m2release/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/miniconda3/envs/m2release/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/minicond

In [2]:
def evaluate_metrics(model, dataloader, text_field, mode="multiple", is_sample=False, beam_size=5, top_k=5, top_p=0.8):
    import itertools
    print(dataloader)
    print(mode)
    model.eval()
    gen = {}
    gts = {}
    with tqdm(desc='evalulateion metrics', unit='it', total=len(dataloader)) as pbar:
        for it, batch in enumerate(iter(dataloader)):
            images = batch['roi_feat']
            caps_gt = batch['cap']
            images = images.to(device)
            with torch.no_grad():
#                 beam_size = 5
                out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<eos>'], beam_size, out_size=1, is_sample=is_sample, top_k=5, top_p=0.8)
#                 if decode == "beam_search":
#                     out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<eos>'], 5, out_size=1, is_sample=False)
#                 elif decode == "top-k_sampling":
#                     out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<eos>'], 1, out_size=1, is_sample=True)
            caps_gen = text_field.decode(out, join_words=False)
            for i, (gts_i, gen_i) in enumerate(zip(caps_gt, caps_gen)):
                gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
                gen['%d_%d' % (it, i)] = [gen_i, ]
                if mode == "multiple":
                    gts['%d_%d' % (it, i)] = gts_i
                elif mode == "single":
                    gts['%d_%d' % (it, i)] = [gts_i[0]]
            pbar.update()

    gts = evaluation.PTBTokenizer.tokenize(gts)
    gen = evaluation.PTBTokenizer.tokenize(gen)
    scores, _ = evaluation.compute_scores(gts, gen, spice=False)
    return scores

In [3]:
device = torch.device('cuda')

In [4]:
# Model
text_field = TextField(init_token='<bos>', eos_token='<eos>', lower=True, tokenize='spacy',
                       remove_punctuation=True, nopoints=False)
text_field.vocab = pickle.load(open('vocab.pkl', 'rb'))

encoder = TransformerEncoder(3, 0, attention_module=ScaledDotProductAttention, attention_module_kwargs={'m': 40})
decoder = MeshedDecoder(len(text_field.vocab), 54, 3, text_field.vocab.stoi['<pad>'])
model = Transformer(text_field.vocab.stoi['<bos>'], encoder, decoder).to(device)

## 1. artpedia

In [7]:
train_myidx = np.load('../Dataset/artpedia/train_myidx.npy')
val_myidx = np.load('../Dataset/artpedia/val_myidx.npy')
test_myidx = np.load('../Dataset/artpedia/test_myidx.npy')

ap_train_dataset = h5py.File("../Dataset/artpedia/ap_train_grid.hdf5", "r")
ap_val_dataset = h5py.File("../Dataset/artpedia/ap_val_grid.hdf5", "r")
ap_test_dataset = h5py.File("../Dataset/artpedia/ap_test_grid.hdf5", "r")
print("loading data: done!!!")

loading data: done!!!


In [8]:
# artpedia dataset
dict_artpedia_test = APeval_Dataset(ap_test_dataset, test_myidx, text_field, max_detections=49, lower=True, remove_punctuation=True, tokenize='spacy')

# artpedia, dataloader
dict_artpedia_test_data_loader = TorchDataLoader(dict_artpedia_test, batch_size=50, collate_fn=lambda x: text_progress2(x))

#### 1.1 artpedia, one caption for evaluation, beam search

In [11]:
# scratch model
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode = "single")

single


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.81it/s]


{'BLEU': [0.0001236055972225921,
  4.035351405369087e-12,
  1.3192351808657412e-14,
  7.681117691112694e-16],
 'ROUGE': 0.00017328068630514444,
 'CIDEr': 2.058933461283359e-06}

In [14]:
# without fine-tuning, off-the-shelf model
data = torch.load('meshed_memory_transformer.pth')
model.load_state_dict(data['state_dict'])

evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single')

single


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.86it/s]


{'BLEU': [0.06869365934607014,
  0.02408617516886492,
  0.009203178196801148,
  0.004077820732889246],
 'ROUGE': 0.1265856198220996,
 'CIDEr': 0.035073845779796387}

In [9]:
# fine-tune on artpedia, one image with one caption for training
data = torch.load('saved_models/artpedia_finetune_singlecap.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single')

single


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.85it/s]


{'BLEU': [0.10770007890173203,
  0.042905798133398614,
  0.021158617232954074,
  0.011994791859808066],
 'ROUGE': 0.16777807810792697,
 'CIDEr': 0.05152375363409006}

In [10]:
# fine-tune on artpedia, one image with multiple captions for training
data = torch.load('saved_models/artpedia_finetune_mulcap.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single')

single


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.95it/s]


{'BLEU': [0.10001649470859435,
  0.04153007097875002,
  0.020453615889150106,
  0.010804099773927376],
 'ROUGE': 0.16959301187547615,
 'CIDEr': 0.04574429996778054}

In [12]:
# fine-tune on artpedia, one image with multiple captions for training   + shuffle
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single')

single


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.90it/s]


{'BLEU': [0.11439772302455756,
  0.049840209225196434,
  0.023219958693446556,
  0.012087941730175456],
 'ROUGE': 0.17763442833720808,
 'CIDEr': 0.04937305896140482}

In [10]:
# beam search.   remove <unk> token
# fine-tune on artpedia, one image with multiple captions for training   + shuffle
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single')

single


evalulateion metrics: 100%|██████████| 329/329 [01:05<00:00,  5.05it/s]


{'BLEU': [0.20352391671520823,
  0.08086088054241472,
  0.03645516117706604,
  0.017655400779776953],
 'ROUGE': 0.18542789079533953,
 'CIDEr': 0.07592433158198265}

In [19]:
# fine-tune on artpedia, one image with multiple captions for training   + shuffle
# generation by top-k sampling    k=5
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single', is_sample=False)

single


evalulateion metrics: 100%|██████████| 329/329 [01:00<00:00,  5.42it/s]


{'BLEU': [0.09333995593432404,
  0.038172705149673934,
  0.01817147245345931,
  0.009833963777214086],
 'ROUGE': 0.16580195687810742,
 'CIDEr': 0.038267756192650784}

In [7]:
# fine-tune on artpedia, one image with multiple captions for training   + shuffle
# generation by top-k sampling    k=10
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single', decode="top-k_sampling")

single


evalulateion metrics: 100%|██████████| 329/329 [01:02<00:00,  5.26it/s]


{'BLEU': [0.13215944908390478,
  0.04599203906981929,
  0.015877252152370982,
  0.005857381895548809],
 'ROUGE': 0.1465819237593196,
 'CIDEr': 0.030852426204639675}

In [18]:
# fine-tune on artpedia, one image with multiple captions for training   + shuffle
# generation by top-k sampling    k=10
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='single', is_sample=True, top_k=10)

single


evalulateion metrics: 100%|██████████| 329/329 [01:02<00:00,  5.23it/s]


{'BLEU': [0.1336230293647883,
  0.0478548036942355,
  0.015488464819317658,
  0.005749882420519693],
 'ROUGE': 0.1460779368214854,
 'CIDEr': 0.027999531132453754}

#### 1.2 artpedia, multiple captions for evaluation, beam search

In [19]:
# *** grid_m2
# without fine-tuning, off-the-shelf model
data = torch.load('saved_models_grid_m2/grid_m2_best.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:05<00:00,  1.27it/s]


{'BLEU': [0.16577379519943217,
  0.06644457162762864,
  0.026349137847583266,
  0.011600651194181912],
 'METEOR': 0.0433855656764746,
 'ROUGE': 0.15793254273513543,
 'CIDEr': 0.023743332859778445}

In [15]:
# fine-tune on artpedia, one image with one caption for training
data = torch.load('saved_models/artpedia_finetune_singlecap.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.94it/s]


{'BLEU': [0.2263770796898909,
  0.10189771635005827,
  0.04723688221258288,
  0.024825597404249923],
 'ROUGE': 0.2044362212159059,
 'CIDEr': 0.03947330064255181}

In [24]:
# *** grid_m2
# fine-tune on artpedia, one image with multiple captions for training
data = torch.load('saved_models/grid_m2_tr_last_15epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field)

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:05<00:00,  1.26it/s]


{'BLEU': [0.25296084904797095,
  0.12311204765152903,
  0.052697501977052766,
  0.025706299968640323],
 'METEOR': 0.06338670711351466,
 'ROUGE': 0.2252853501235557,
 'CIDEr': 0.03524534922558475}

In [16]:
# fine-tune on artpedia, one image with multiple captions for training
data = torch.load('saved_models/artpedia_finetune_mulcap.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field)

multiple


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.95it/s]


{'BLEU': [0.21496088729295892,
  0.10277365798269977,
  0.04822575058710956,
  0.024778542489085762],
 'ROUGE': 0.2091605244824866,
 'CIDEr': 0.03343695862148488}

In [22]:
# fine-tune on artpedia, one image with multiple captions for training      shuffle
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field)

multiple


evalulateion metrics: 100%|██████████| 329/329 [01:03<00:00,  5.18it/s]


{'BLEU': [0.23290164145038766,
  0.11628312648636245,
  0.052702077361140964,
  0.02689678159915748],
 'ROUGE': 0.21914811563939154,
 'CIDEr': 0.03148247854508848}

In [8]:
# beam search   remove unk
# fine-tune on artpedia, one image with multiple captions for training      shuffle
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, is_sample=False)

multiple


evalulateion metrics: 100%|██████████| 329/329 [01:05<00:00,  5.00it/s]


{'BLEU': [0.33960716243253547,
  0.16180978856929548,
  0.0747107343165195,
  0.035472046837868663],
 'ROUGE': 0.22451221642237176,
 'CIDEr': 0.06145132500410184}

In [11]:
# *** grid_m2
# beam search   remove unk
# fine-tune on artpedia, one image with multiple captions for training      shuffle
data = torch.load('saved_models/grid_m2_tr_last_15epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, is_sample=False)

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:08<00:00,  1.17s/it]


{'BLEU': [0.35762850093836457,
  0.17222325774134367,
  0.08031597949946138,
  0.0389859827653499],
 'METEOR': 0.08842069618105143,
 'ROUGE': 0.23075065821853866,
 'CIDEr': 0.06807426873269039}

## 2. semart

In [5]:
sa_test_csv = pd.read_csv("../Dataset/SemArt/prediction_csvs/semart_test_prediction.csv")
sa_test_csv = sa_test_csv[sa_test_csv['predictioin']==0]
test_roi_feats = h5py.File("../Dataset/SemArt/sa_test_grid.hdf5", "r")
test_img_names = np.unique(sa_test_csv['img_name'].to_numpy())
test_img_caps_map = json.load(open('../Dataset/SemArt/test_img_caps_map.json'))

dict_semart_test = SAeval_Dataset(sa_test_csv, test_img_names, test_img_caps_map, test_roi_feats, text_field, max_detections=49, lower=True, remove_punctuation=True, tokenize='spacy')
dict_semart_test_data_loader = TorchDataLoader(dict_semart_test, batch_size=10,
                                  collate_fn=lambda x: text_progress2(x))

#### 2.1 semart, multiple captions for evaluation

In [19]:
# off-the-shelf model  ***
data = torch.load('saved_models/saved_models_grid_m2/grid_m2_best.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:15<00:00,  3.89it/s]


{'BLEU': [0.11533953842197162,
  0.039782848723446396,
  0.014352327712648954,
  0.005583332053985084],
 'METEOR': 0.037317946222108496,
 'ROUGE': 0.1407143406791696,
 'CIDEr': 0.020143350427409674}

In [20]:
# fine-tune on SemArt
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_10epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:21<00:00,  2.90it/s]


{'BLEU': [0.21463416441072608,
  0.1058966925124788,
  0.05256788929370499,
  0.030818367459357468],
 'METEOR': 0.06387810691304108,
 'ROUGE': 0.21598327906765846,
 'CIDEr': 0.0599095238125901}

In [21]:
# fine-tune on SemArt
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_13epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:15<00:00,  3.89it/s]


{'BLEU': [0.19316501521892346,
  0.09220240830526882,
  0.04682501367553965,
  0.028436043589034024],
 'METEOR': 0.06169974966082502,
 'ROUGE': 0.2065315798846974,
 'CIDEr': 0.06704352388762456}

In [13]:
# fine-tune on SemArt
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_12epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:20<00:00,  2.98it/s]


{'BLEU': [0.31310133603043044,
  0.153552917771243,
  0.07456135060987139,
  0.042422365680723065],
 'METEOR': 0.08446232128562489,
 'ROUGE': 0.23355387825854562,
 'CIDEr': 0.10082804867346877}

In [23]:
# fine-tune on SemArt
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_11epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:15<00:00,  3.88it/s]


{'BLEU': [0.19546150334262233,
  0.09774340995591328,
  0.051427166367178916,
  0.03158941865261445],
 'METEOR': 0.06383502717845177,
 'ROUGE': 0.2160521035677449,
 'CIDEr': 0.07208048787412573}

In [6]:
# fine-tune on SemArt     remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_11epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:20<00:00,  2.99it/s]


{'BLEU': [0.3136397727171599,
  0.1539397732792538,
  0.07414077612473546,
  0.04129592891984856],
 'METEOR': 0.08531230505769902,
 'ROUGE': 0.23569184859501466,
 'CIDEr': 0.09515643902965266}

In [8]:
# fine-tune on SemArt     remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_10epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:20<00:00,  3.01it/s]


{'BLEU': [0.3112334920154185,
  0.15523951377741668,
  0.07910016576933099,
  0.04503895024022269],
 'METEOR': 0.08413479323419014,
 'ROUGE': 0.23497037598933013,
 'CIDEr': 0.0950346285443715}

In [9]:
# fine-tune on SemArt     remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_12epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:20<00:00,  3.00it/s]


{'BLEU': [0.31310133603043044,
  0.153552917771243,
  0.07456135060987139,
  0.042422365680723065],
 'METEOR': 0.08446232128562489,
 'ROUGE': 0.23355387825854562,
 'CIDEr': 0.10082804867346877}

In [12]:
# fine-tune on SemArt     remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2/sa_gridm2_sa_last_13epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 62/62 [00:20<00:00,  3.00it/s]


{'BLEU': [0.3017095870446208,
  0.1441650904246923,
  0.07032731256466737,
  0.039046321719393526],
 'METEOR': 0.08222837145922787,
 'ROUGE': 0.22438646854582237,
 'CIDEr': 0.0967136906762575}